In [ ]:
from collections import defaultdict
import csv
import json
from glob import glob
import os
import numpy as np
import pandas as pd
import partridge as ptg
from scipy.spatial.distance import directed_hausdorff
import shapefile

In [ ]:
# shp_path uses only ONE json file
# gtfs_path uses one or more GTFS files
# in this case there are more than one GTFS as an example:
# '/Users/Charlie/Downloads/output-0b06ab05-b943-4ddc-92e7-33b62ce82bf4/*'
shp_path = '/Users/santiagotoso/GoogleDrive/Master/Python/Partridge/json.geojson'
gtfs_path = '/Users/santiagotoso/GoogleDrive/Master/Python/Partridge/GTFS_for_test'
#feed_paths = list(glob(gtfs_path))

In [ ]:
#len(feed_paths)

In [ ]:
# feed_paths[0]

In [ ]:
with open(shp_path) as f:
    fc = json.load(f)

feed = ptg.load_raw_feed(gtfs_path)

In [ ]:
feed

In [ ]:
S = []
for feature in fc['features']:
    points = feature['geometry']['coordinates']
    
    bad_points = 0
    while points[0] == points[-1]:
        bad_points += 1
        points = points[:-1]

    if bad_points:
        print('cleaned up {} wrapped points'.format(bad_points))

    S.append(np.array(points))

In [ ]:
import math

def same_direction(p, s):
    ''' Return True if the pattern and shape point in similar directions
    '''
    p_theta = math.atan2(p[-1][1] - p[0][1], p[-1][0] - p[0][0])
    s_theta = math.atan2(s[-1][1] - s[0][1], s[-1][0] - s[0][0])
    cosine = math.cos(p_theta - s_theta)
    
    # cosine is positive when p and s point in the same direction
    return bool(cosine > 0)

In [ ]:
def match_shapes_to_feed(inpath, outpath, S):
    print('matching feed', inpath)
    feed = ptg.load_raw_feed(inpath)

    pattern_by_trip_id = {}
    patterns = set()
    for trip_id, stop_times in feed.stop_times.sort_values('stop_sequence').groupby('trip_id'):
        pattern = tuple(stop_times.stop_id)
        patterns.add(pattern)
        pattern_by_trip_id[trip_id] = pattern

    patterns = list(patterns)

    stops = {}
    for _, stop in feed.stops.iterrows():
        stops[stop.stop_id] = (stop.stop_lon, stop.stop_lat)
    
    P = []
    for pattern in patterns:
        P.append([stops[stop_id] for stop_id in pattern])
    
    C = []
    
    # degrees_per_foot tells us how strict we wanna be when matching shapes
    # bigger numbers imply more tolerance
    degrees_per_foot = 3.788e-06
    max_dist = 350 * degrees_per_foot
    for p in P:
        best_idx = None
        best_dist = float('inf')
        for j, s in enumerate(S):
            if not same_direction(p, s):
                continue
            dist = directed_hausdorff(p, s)[0]
            if dist > max_dist:
                continue
            if dist < best_dist:
                best_dist = dist
                best_idx = j

        C.append(best_idx)
    
    shapes = {
        'shape_id': [],
        'shape_pt_sequence': [],
        'shape_pt_lat': [],
        'shape_pt_lon': [],
    }

    shape_id_by_pattern = {}
    for shape_id, pattern in enumerate(patterns):
        shape_id_by_pattern[pattern] = shape_id
        if C[shape_id] is None:
            continue
        shape_points = S[C[shape_id]]
        for shape_pt_sequence, (shape_pt_lon, shape_pt_lat) in enumerate(shape_points, start=1):
            shapes['shape_id'].append(shape_id)
            shapes['shape_pt_sequence'].append(shape_pt_sequence)
            shapes['shape_pt_lon'].append(shape_pt_lon)
            shapes['shape_pt_lat'].append(shape_pt_lat)
        
    output_feed = ptg.raw_feed(inpath)
    
    def get_shape_id_by_trip_id(trip_id):
        pattern = pattern_by_trip_id[trip_id]
        return shape_id_by_pattern[pattern]

    output_feed.trips['shape_id'] = output_feed.trips.trip_id.apply(get_shape_id_by_trip_id)
    
    sdf = pd.DataFrame(shapes)

    output_feed.shapes['shape_id'] = sdf.shape_id
    output_feed.shapes['shape_pt_sequence'] = sdf.shape_pt_sequence
    output_feed.shapes['shape_pt_lon'] = sdf.shape_pt_lon
    output_feed.shapes['shape_pt_lat'] = sdf.shape_pt_lat
    
    print('writing feed', outpath)
    ptg.writers.write_feed_dangerously(output_feed, outpath)

In [ ]:
for inpath in gtfs_path:
    fname = os.path.basename(inpath)
    dirname = os.path.dirname(inpath)
    basename, ext = os.path.splitext(fname)
    outpath = os.path.join(dirname, 'outputs', basename + '_shapes'+ ext)
    
    match_shapes_to_feed(inpath, outpath, S)
